In [2]:
from cassandra.cluster import Cluster
import pandas as pd


In [11]:

# Connect to Cassandra outside of callback function
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()
session.set_keyspace('dmsb_tfm')


query = "SELECT * FROM candlesticks LIMIT 300"

result = session.execute(query)
rows = [row._asdict() for row in result]
df = pd.DataFrame(rows)

# We filter only to have the newest data for each time
idx = df.groupby('start_time')['event_time'].idxmax()
df = df.loc[idx]
df = df.sort_values(by='start_time', ascending=False)
print(len(df))

30


In [14]:
df_kline_closed = df[df['kline_closed'] == True]

df_close = df_kline_closed[['start_time', 'close']]
df_open = df_kline_closed[['start_time', 'open']]
df_high = df_kline_closed[['start_time', 'high']]
df_low = df_kline_closed[['start_time', 'low']]

,start_time,low
0,2024-02-14 13:30:00,48276.99
3,2024-02-14 13:29:00,48246.19
10,2024-02-14 13:28:00,48249.25
19,2024-02-14 13:27:00,48251.03
24,2024-02-14 13:26:00,48232.51


In [25]:
df = (df_close['close']).tolist()
df

[48276.99,
 48255.5,
 48264.26,
 48251.03,
 48262.75,
 48270.99,
 48218.66,
 48182.44,
 48200.0,
 48145.02,
 48145.78,
 48159.99,
 48175.83,
 48159.06,
 48172.85,
 48160.0,
 48116.39,
 48130.11,
 44630.74,
 44631.89,
 44617.77,
 44651.56,
 44665.56,
 44674.07,
 44681.94,
 44673.73,
 44674.71,
 44661.85,
 44666.23,
 44657.79]

In [39]:
from statsmodels.tsa.arima.model import ARIMA


close_model = ARIMA(df_close['close'].tolist(), order = (4,1,0))
open_model = ARIMA(df_open['open'].tolist(), order = (4,1,0))
high_model = ARIMA(df_high['high'].tolist(), order = (4,1,0))
low_model = ARIMA(df_low['low'].tolist(), order = (4,1,0))

close_model_fit = close_model.fit()
open_model_fit = open_model.fit()
high_model_fit = high_model.fit()
low_model_fit = low_model.fit()

close_prediction = close_model_fit.forecast()
open_prediction = open_model_fit.forecast()
high_prediction = high_model_fit.forecast()
low_prediction = low_model_fit.forecast()

high_prediction

array([44677.69181336])

In [48]:
df_test = df_kline_closed.head(3)
df_test.head()

,symbol,event_time,base_asset_volume,close,close_time,high,kline_closed,low,number_of_trades,open,quote_asset_volume,start_time
3,BTCEUR,2024-02-14 13:30:00,0.08709,48255.50,2024-02-14 13:29:59,48268.22,True,48246.19,12,48246.19,4202.580628,2024-02-14 13:29:00
10,BTCEUR,2024-02-14 13:29:00,0.08861,48264.26,2024-02-14 13:28:59,48264.46,True,48249.25,14,48264.46,4275.959418,2024-02-14 13:28:00
19,BTCEUR,2024-02-14 13:28:00,0.13001,48251.03,2024-02-14 13:27:59,48268.97,True,48251.03,16,48256.20,6273.788420,2024-02-14 13:27:00


In [52]:
pd.Timedelta(minutes=2)

Timedelta('0 days 00:02:00')

In [56]:
df_test['start_time']

3    2024-02-14 13:29:00
10   2024-02-14 13:28:00
19   2024-02-14 13:27:00
Name: start_time, dtype: datetime64[ns]

In [73]:
# Add one minute to each timestamp
df_new = df_test['start_time'].iloc[0]
df_new

Timestamp('2024-02-14 13:29:00')

In [7]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({'flag': [False, False, False, True, False, False, True, False, True]})

# Find the index where the first True occurs
first_true_index = df[df['flag']].index[0]

# Split the DataFrame into two parts
df_before_true = df.iloc[:first_true_index]
df_after_true = df.iloc[first_true_index:]

# Keep only the True values in the second part
df_after_true = df_after_true[df_after_true['flag']]

# Concatenate the two parts
filtered_df = pd.concat([df_before_true, df_after_true])

print(filtered_df)


    flag
0  False
1  False
2  False
3   True
6   True
8   True
